In [1]:
import graphlab

In [2]:
products = graphlab.SFrame('amazon_baby.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to dhavalp576@gmail.com and will expire on May 27, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\DHAVAL~1\AppData\Local\Temp\graphlab_server_1497158565.log.0


In [3]:
products.head()

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


In [4]:
products[269]

{'name': 'The First Years Massaging Action Teether',
 'rating': 5.0,
 'review': 'A favorite in our house!'}

In [5]:
def remove_punctuation(text):
    import string
    return text.translate(None, string.punctuation) 

products['review_clean'] = products['review'].apply(remove_punctuation)

In [7]:
products[269]

{'name': 'The First Years Massaging Action Teether',
 'rating': 5.0,
 'review': 'A favorite in our house!',
 'review_clean': 'A favorite in our house'}

In [19]:
products[products['review'] == '']

name,review,rating,review_clean
SoftPlay Twinkle TwinkleElmo A Bedtime Book ...,,5.0,
Our Baby Girl Memory Book,,5.0,
"Summer Infant, UltimateTraining Pad - Twin ...",,5.0,
Safety 1st Deluxe 4-in-1Bath Station ...,,1.0,
Northstate SuperyardPlaygate Light Gray ...,,5.0,
Munchkin Mozart MagicCube ...,,5.0,
"Graco TotBloc Pack 'NPlay with Carry Bag, ...",,5.0,
"Graco TotBloc Pack 'NPlay with Carry Bag, ...",,5.0,
Prince Lionheart TableEdge Guard with 4 ...,,2.0,
BABYBJORN Toilet Trainer- White/Red ...,,5.0,


In [25]:
products = products.fillna('review','')  # fill in N/A's in the review column
products = products.fillna('review_clean','')  # fill in N/A's in the review column

In [26]:
products

name,review,rating,review_clean,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,it came early and was notdisappointed i love ...,1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,Very soft and comfortableand warmer than it ...,1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,This is a product wellworth the purchase I ...,1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,All of my kids have criednonstop when I tried to ...,1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,When the Binky Fairy cameto our house we didnt ...,1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,Lovely book its boundtightly so you may no ...,1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,Perfect for new parentsWe were able to keep ...,1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,A friend of mine pinnedthis product on Pinte ...,1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,This has been an easy wayfor my nanny to record ...,1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,I love this journal andour nanny uses it ...,1


In [27]:
products = products[products['rating'] != 3]

In [28]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)

In [29]:
train_data, test_data = products.random_split(.8, seed=1)

In [30]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
    # Use this token pattern to keep single-letter words
# First, learn vocabulary from the training data and assign columns to words
# Then convert the training data into a sparse matrix
train_matrix = vectorizer.fit_transform(train_data['review_clean'])
# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform(test_data['review_clean'])

In [32]:
train_matrix.shape

(133416, 121712)

In [35]:
from sklearn import linear_model

In [36]:
sentimental_model = linear_model.LogisticRegression()

In [37]:
sentimental_model.fit(train_matrix, train_data['sentiment'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [45]:
import pandas as pd
coefficients = pd.DataFrame(sentimental_model.coef_).transpose()

In [50]:
positive_coef = coefficients[coefficients>=0].dropna().count()
negative_coef = coefficients[coefficients<0].dropna().count()

In [51]:
print positive_coef
print negative_coef

0    85929
dtype: int64
0    35783
dtype: int64


In [52]:
sample_test_data = test_data[10:13]
print sample_test_data

+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|   Our Baby Girl Memory Book   | Absolutely love it and all... |  5.0   |
| Wall Decor Removable Decal... | Would not purchase again o... |  2.0   |
| New Style Trailing Cherry ... | Was so excited to get this... |  1.0   |
+-------------------------------+-------------------------------+--------+
+-------------------------------+-----------+
|          review_clean         | sentiment |
+-------------------------------+-----------+
| Absolutely love it and all... |     1     |
| Would not purchase again o... |     -1    |
| Was so excited to get this... |     -1    |
+-------------------------------+-----------+
[3 rows x 5 columns]



In [55]:
sample_test_data[0]['review']

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

In [56]:
sample_test_data[1]['review']

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

In [58]:
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'])
scores = sentimental_model.decision_function(sample_test_matrix)
print scores

[  5.60690029  -3.14219746 -10.40284605]


In [60]:
pd.Series(scores).apply(lambda rating : +1 if rating > 0 else -1)

0    1
1   -1
2   -1
dtype: int64

In [61]:
sentimental_model.predict(sample_test_matrix)

array([ 1, -1, -1], dtype=int64)

In [62]:
import numpy as np

In [63]:
np.exp(scores)

array([  2.72298882e+02,   4.31877903e-02,   3.03459939e-05])

In [65]:
print (1/(1 + np.exp(-scores)))

[  9.96341002e-01   4.13998234e-02   3.03450731e-05]


In [104]:
sample_test_matrix = vectorizer.transform(test_data['review_clean'])
scores_test = sentimental_model.predict_proba(test_matrix)
test_data['predicted_sentiment'] = sentimental_model.predict(test_matrix)

In [105]:
test_data.head()

name,review,rating,review_clean,sentiment
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,This has been an easy wayfor my nanny to record ...,1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,I love this journal andour nanny uses it ...,1
Nature's Lullabies FirstYear Sticker Calendar ...,"I love this littlecalender, you can keep ...",5.0,I love this littlecalender you can keep ...,1
Nature's Lullabies SecondYear Sticker Calendar ...,"I had a hard time findinga second year calendar, ...",5.0,I had a hard time findinga second year calendar ...,1
"Lamaze Peekaboo, I LoveYou ...","One of baby's first andfavorite books, and i ...",4.0,One of babys first andfavorite books and it is ...,1
"Lamaze Peekaboo, I LoveYou ...",My son loved this book asan infant. It was ...,5.0,My son loved this book asan infant It was per ...,1
"Lamaze Peekaboo, I LoveYou ...",Our baby loves this book& has loved it for a ...,5.0,Our baby loves this bookhas loved it for a while ...,1
"SoftPlay Giggle JiggleFunbook, Happy Bear ...",This bear is absolutelyadorable and I would ...,2.0,This bear is absolutelyadorable and I would ...,-1
SoftPlay Peek-A-BooWhere's Elmo A Childr ...,I bought two for recentbaby showers! The book ...,5.0,I bought two for recentbaby showers The boo ...,1
Baby's First Year UndatedWall Calendar with ...,I searched high and lowfor a first year cale ...,5.0,I searched high and lowfor a first year cale ...,1


In [106]:
print scores_test[7]

[ 0.19167803  0.80832197]


In [107]:
scores_test.shape

(33336L, 2L)

In [108]:
test_data.shape

(33336, 7)

In [109]:
test_data['proba'] = scores_test[:,1]

In [110]:
test_data.sort('proba', ascending=False).print_rows(num_rows=20)

+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
| Graco FastAction Fold Jogg... | Graco's FastAction Jogging... |  5.0   |
| Graco Pack 'n Play Element... | My husband and I assembled... |  4.0   |
| Freemie Hands-Free Conceal... | I absolutely love this pro... |  5.0   |
| Baby Einstein Around The W... | I am so HAPPY I brought th... |  5.0   |
| Evenflo 6 Pack Classic Gla... | It's always fun to write a... |  5.0   |
| Fisher-Price Cradle 'N Swi... | My husband and I cannot st... |  5.0   |
| Mamas &amp; Papas 2014 Urb... | After much research I purc... |  4.0   |
| Britax 2012 B-Agile Stroll... | [I got this stroller for m... |  4.0   |
| Simple Wishes Hands-Free B... | I just tried this hands fr... |  5.0   |
| Diono RadianRXT Convertibl... | I bought this seat for my ... |  5.0   |
| Roan Rocco Classic Pram

In [111]:
test_data.sort('proba').print_rows(num_rows=20)

+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
| Fisher-Price Ocean Wonders... | We have not had ANY luck w... |  2.0   |
| Levana Safe N'See Digital ... | This is the first review I... |  1.0   |
| Safety 1st Exchangeable Ti... | I thought it sounded great... |  1.0   |
| Adiri BPA Free Natural Nur... | I will try to write an obj... |  2.0   |
| VTech Communications Safe ... | This is my second video mo... |  1.0   |
| The First Years True Choic... | Note: we never installed b... |  1.0   |
| Safety 1st High-Def Digita... | We bought this baby monito... |  1.0   |
| Cloth Diaper Sprayer--styl... | I bought this sprayer out ... |  1.0   |
| Motorola Digital Video Bab... | DO NOT BUY THIS BABY MONIT... |  1.0   |
| Philips AVENT Newborn Star... | It's 3am in the morning an... |  1.0   |
| Cosco Alpha Omega Elite

In [119]:
correct_predict = test_data[test_data['sentiment'] == test_data['predicted_sentiment']].num_rows()

In [120]:
total_data = test_data.num_rows()

In [124]:
accuracy = correct_predict / float(total_data)
print accuracy

0.932265418766


In [125]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [126]:
len(significant_words)

20

In [128]:
vectorizer_word_subset = CountVectorizer(vocabulary=significant_words) # limit to 20 words
train_matrix_word_subset = vectorizer_word_subset.fit_transform(train_data['review_clean'])
test_matrix_word_subset = vectorizer_word_subset.transform(test_data['review_clean'])

In [129]:
simple_model = linear_model.LogisticRegression()
simple_model.fit(train_matrix_word_subset, train_data['sentiment'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [130]:
simple_model_coef_table = pd.DataFrame({'word':significant_words,
                                         'coefficient':simple_model.coef_.flatten()})

In [132]:
simple_model_coef_table.sort('coefficient')

C:\Users\Dhaval-PC\Anaconda2\envs\gl-env\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,coefficient,word
14,-2.348298,disappointed
19,-2.109331,return
13,-2.033699,waste
10,-1.651576,broke
17,-0.898031,money
15,-0.621169,work
12,-0.511380,even
18,-0.362167,would
16,-0.320556,product
11,-0.209563,less


In [134]:
train_data['predicated_sentiment'] = sentimental_model.predict(train_matrix)
correct_predict_train = train_data[train_data['sentiment'] == train_data['predicated_sentiment']].num_rows()
total_data_train = train_data.num_rows()
accuracy_train = correct_predict_train / float(total_data_train)
print accuracy_train

0.967964861786


In [136]:
train_data['simple_predicated_sentiment'] = simple_model.predict(train_matrix_word_subset)
correct_predict_train_simple = train_data[train_data['sentiment'] == train_data['simple_predicated_sentiment']].num_rows()
total_data_train = train_data.num_rows()
accuracy_train_simple = correct_predict_train_simple / float(total_data_train)
print accuracy_train_simple

0.866822570007


In [137]:
test_data['simple_predicated_sentiment'] = simple_model.predict(test_matrix_word_subset)
correct_predict_test_simple = test_data[test_data['sentiment'] == test_data['simple_predicated_sentiment']].num_rows()
total_data_test = test_data.num_rows()
accuracy_test_simple = correct_predict_test_simple / float(total_data_test)
print accuracy_test_simple

0.869360451164


In [138]:
mcc_count = test_data[test_data['sentiment'] == 1].num_rows()
total_data_test = test_data.num_rows()
accuracy_mcc = mcc_count / float(total_data_test)
print accuracy_mcc

0.842782577394
